#### Cleaning Players Data

In [1]:
import pandas as pd
import regex as re
from datetime import datetime    
from unidecode import unidecode

In [2]:
players = pd.read_csv("2022.csv")

In [3]:
players["Year"] = 2022

In [4]:
#Reformatting how club are written
players["Club"] =[unidecode(w) for w in players["Club"]]

players["Player"] =[unidecode(w) for w in players["Player"]]

In [5]:
players["Player"] = [re.sub(' +', ' ', w) for w in players["Player"]]

#### Transforming Values

In [6]:
#Getting the number of days left on contract
Contract_end = []

for w in players["Contract"] : 
    try:
        length = datetime.strptime(str(w), '%b %d, %Y') - datetime(2022,7,1,0,0,0,0)
        Contract_end.append(length.days)  
    except (ValueError, AttributeError): 
        Contract_end.append("NA")

players["Contract"] = Contract_end

In [7]:
#Getting the Age
Age = []

for row in players["Date of Birth"] :
    try : 
        Age.append(re.search(r'\((.*?)\)',row[row.rindex(',')+1:]).group(1))
    except (ValueError, AttributeError): 
        Age.append("NA")
        
players["Date of Birth"] = Age

players = players.rename(columns={'Date of Birth': 'Age'})

In [8]:
#Number of days since they joined the club
Longetivity = []
for w in players["Join"] : 
    try:
        length = datetime(2022,7,1,0,0,0,0) - datetime.strptime(str(w), '%b %d, %Y')
        Longetivity.append(length.days) 
    except (ValueError, AttributeError): 
        Longetivity.append("NA")

players["Join"] = Longetivity

In [9]:
#Converting to numeric
players[['Age', 'Join', 'Contract']] = players[['Age', 'Join', 'Contract']].apply(pd.to_numeric, errors='coerce', axis=1)

In [10]:
#Transforming the market value column
def value_to_float(x):
    x = str(x).replace('€', '')
    if 'k' in x:
        return float(x.replace('k', '')) / 1000
    if 'm' in x:
        return float(x.replace('m', ''))
    return 0

In [11]:
#Formatting
players['Market Value'] = players['Market Value'].apply(value_to_float)

In [12]:
names = ["Alisson", "Neymar", "Denilson", "Vitinho", "Martinelli", "Hulk", "Koke", "Raphinha", "Casemiro", "Marcelo", "Jorginho", "Ederson", "Willian", "Antony", "Marquinhos", "Thiago"]

for name in names : 
    row = players.loc[players["Player"] == name,].sort_values('Market Value', ascending=False).drop_duplicates('Player').iloc[:,:]
    players = players.loc[players["Player"] != name,]
    players = pd.concat([players, row], axis=0, ignore_index=True)

In [13]:
#In case of January transfer/ loan keep the original club
players = players.sort_values('Join', ascending=False).drop_duplicates('Player').sort_index()

#### Dropping Columns

In [14]:
#Contract Column dropped even though I could be kept however contract is new contract
players = players.drop([ "Season", "Club Link", "Contract"], axis=1)

#### Club Cleaning

In [15]:
players["Club"] = [l.replace('FC', '') for l in players["Club"]]

In [16]:
players["Club"] = [l.replace('Wolverhampton Wanderers', 'Wolves') for l in players["Club"]]

In [17]:
players["Club"] = [l.replace('Olympique', '') for l in players["Club"]]

In [18]:
players["Club"] = [l.replace('Hertha BSC', 'Hertha') for l in players["Club"]]

In [19]:
players.to_csv('Players_2022.csv', encoding='utf-8')